In [1]:
import sys
import os

project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.append(project_root)

%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from src.features import build_features
from src.visualization import visualize

sns.set_style()
plt.rcParams["figure.figsize"] = (12, 6)

DIAGS_RELEVANTES = [
    "C33X",
    "C340",
    "C341",
    "C342",
    "C343",
    "C381",
    "C384",
    "C450",
    "C780",
    "C782",
    "D143",
    "D381",
    "E848",
    "I080",
    "I081",
    "I340",
    "I350",
    "I351",
    "I352",
    "I420",
    "I456",
    "I472",
    "I495",
    "I710",
    "I712",
    "J398",
    "J47X",
    "J679",
    "J841",
    "J848",
    "J849",
    "J860",
    "J869",
    "J90X",
    "J91X",
    "J931",
    "J955",
    "J980",
    "M348",
    "Q211",
    "Q231",
    "Q676",
    "T820",
    "T821",
    "Z450",
]

DIAGS_FALTANTES_PULMON = ["C349", "C348"]

DIAGS_RELEVANTES += DIAGS_FALTANTES_PULMON

In [3]:
# Carga los egresos nacionales
egresos_nacionales = pd.read_csv(
    "../data/processed/ranking_nacional_egresos.csv", sep=";", encoding="latin-1"
)

In [4]:
# Filtra solamente los diagnosticos mas relevantes del Torax
egresos_mas_relevantes = (
    egresos_nacionales[egresos_nacionales["DIAG1"].isin(DIAGS_RELEVANTES)]
    .query("ANO_EGRESO >= 2017")
    .copy()
)

egresos_mas_relevantes["DIAG1"] = egresos_mas_relevantes["DIAG1"].replace(
    ["C340", "C341", "C342", "C343", "C348", "C349"], "C34"
)

In [5]:
# Obtiene cantidad de pacientes, egresos y dias de estada por diagnostico
resumen_egresos_mas_relevantes = pd.pivot_table(
    egresos_mas_relevantes,
    index="DIAG1",
    columns="ANO_EGRESO",
    values=["n_pacientes_distintos", "n_egresos", "dias_estada_totales"],
    aggfunc="sum",
    fill_value=0,
).sort_index()

In [6]:
# Importa los casos teoricos por cada problema de salud
casos_teoricos = pd.read_excel("../data/interim/casos_teoricos_diagnosticos.xlsx")
casos_teoricos["Diagnostico"] = casos_teoricos["Diagnostico"].str.split("-").str[0].str.strip()
casos_teoricos = casos_teoricos.set_index("Diagnostico")

# Obtiene los casos en el estrato nacional
casos_nacionales = casos_teoricos.query("Estrato == 'Pais' and Estadística == 'Incidencia'").copy()
casos_nacionales_del_2017_al_2020 = casos_nacionales[[f"Poblacion {i}" for i in range (2017, 2021)]]

In [20]:
# Une la cantidad de casos teoricos y reales
union_de_pacientes_teoricos_y_vistos = casos_nacionales_del_2017_al_2020.merge(
    resumen_egresos_mas_relevantes["n_pacientes_distintos"],
    how="left",
    left_index=True,
    right_index=True,
)

# Renombra las columnas
niveles_columnas = [
    ("poblacion_teorica", "2017"),
    ("poblacion_teorica", "2018"),
    ("poblacion_teorica", "2019"),
    ("poblacion_teorica", "2020"),
    ("poblacion_vista", "2017"),
    ("poblacion_vista", "2018"),
    ("poblacion_vista", "2019"),
    ("poblacion_vista", "2020"),
]

union_de_pacientes_teoricos_y_vistos.columns = pd.MultiIndex.from_tuples(niveles_columnas)

# Calcula la cantidad de cumplimiento de los casos teoricos
porcentaje_de_atencion_casos_teoricos = (
    union_de_pacientes_teoricos_y_vistos["poblacion_vista"].sum(axis=1)
    / union_de_pacientes_teoricos_y_vistos["poblacion_teorica"].sum(axis=1)
)

In [24]:
proyeccion_de_casos_a_atender = casos_nacionales[[f"Poblacion {i}" for i in range(2017, 2036)]].mul(
    porcentaje_de_atencion_casos_teoricos, axis=0
)

In [34]:
proyeccion_de_casos_a_atender

,Poblacion 2017,Poblacion 2018,Poblacion 2019,Poblacion 2020,Poblacion 2021,Poblacion 2022,Poblacion 2023,Poblacion 2024,Poblacion 2025,Poblacion 2026,Poblacion 2027,Poblacion 2028,Poblacion 2029,Poblacion 2030,Poblacion 2031,Poblacion 2032,Poblacion 2033,Poblacion 2034,Poblacion 2035
Diagnostico,,,,,,,,,,,,,,,,,,,
C33X,14.105728,14.360142,14.632628,14.901502,15.070022,15.185048,15.286385,15.382486,15.474825,15.563548,15.648542,15.729623,15.806636,15.879434,15.949206,16.014975,16.076664,16.134227,16.187660
C34,2533.437354,2579.131043,2628.070479,2676.361124,2706.627950,2727.286961,2745.487523,2762.747562,2779.331989,2795.266936,2810.532048,2825.094450,2838.926359,2852.001092,2864.532392,2876.344752,2887.424281,2897.762727,2907.359539
C381,37.696342,38.376242,39.104437,39.822979,40.273335,40.580731,40.851547,41.108368,41.355137,41.592241,41.819379,42.036060,42.241872,42.436418,42.622878,42.798640,42.963498,43.117330,43.260126
C450,47.910834,48.774965,49.700478,50.613722,51.186110,51.576801,51.920999,52.247410,52.561045,52.862397,53.151081,53.426477,53.688057,53.935319,54.172303,54.395691,54.605221,54.800735,54.982224
C780,336.348647,342.415113,348.912496,355.323743,359.342081,362.084849,364.501223,366.792731,368.994542,371.110125,373.136779,375.070138,376.906514,378.642364,380.306067,381.874320,383.345280,384.717851,385.991960
C782,165.620701,168.607876,171.807238,174.964186,176.942847,178.293407,179.483249,180.611605,181.695794,182.737524,183.735464,184.687465,185.591711,186.446457,187.265677,188.037897,188.762209,189.438074,190.065455
E848,39.209183,38.405346,38.787780,39.597690,40.062785,40.119130,39.983498,39.764193,39.485001,39.155369,38.783900,38.379705,37.951217,37.505015,37.340199,36.974635,36.606540,36.244518,35.898354
I051,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
I340,335.619041,341.672347,348.155636,354.552976,358.562597,361.299415,363.710548,365.997085,368.194120,370.305114,372.327371,374.256536,376.088929,377.821014,379.481108,381.045959,382.513728,383.883322,385.154667


In [28]:
resumen_egresos_mas_relevantes["n_pacientes_distintos"]

ANO_EGRESO,2017,2018,2019,2020
DIAG1,,,,
C33X,19,13,15,11
C34,2573,2785,2785,2274
C381,34,35,51,35
C384,38,40,49,42
C450,53,54,49,41
C780,339,347,385,312
C782,136,176,192,177
D143,165,189,150,108
D381,512,597,608,532
